In [13]:
import sys
from pathlib import Path
import os

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
if os.path.exists(f"{root_dir}/.env"):
    settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /Users/erik/Code/Skola/ID2223/mlfs-book
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [14]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [15]:
project = hopsworks.login()
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON_A").value
location = json.loads(location_str)


country=location['country']
city=location['city']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

2025-11-18 12:01:23,998 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-18 12:01:24,005 INFO: Initializing external client
2025-11-18 12:01:24,006 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-18 12:01:25,485 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279161


'{"country": "Germany", "city": "Regensburg", "street": "Pfeilstra\\u00dfe", "aqicn_url": "https://api.waqi.info/feed/A66394", "latitude": 49.036, "longitude": 12.112}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [16]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality_a',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather_a',
    version=1,
)

urls = air_quality_fg.read().url.unique().tolist()
streets = air_quality_fg.read().street.unique().tolist()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.20s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.92s) 


In [17]:
urls

['https://api.waqi.info/feed/A191545',
 'https://api.waqi.info/feed/A474136',
 'https://api.waqi.info/feed/A469291',
 'https://api.waqi.info/feed/A66394']

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [18]:
import requests
import pandas as pd
aq_today_df = pd.DataFrame()
for url, street in zip(urls, streets):
    print(f"Fetching data for url: {url}")
    
    
    temp = util.get_pm25(url, country, city, street, today, AQICN_API_KEY, url)
    aq_today_df = pd.concat([aq_today_df, temp])

aq_today_df


Fetching data for url: https://api.waqi.info/feed/A191545
Fetching AQI data from URL with ID: https://api.waqi.info/feed/A191545/?token=f8c9b0aad7e624da4286392f37a47c05f9322983
Fetching data for url: https://api.waqi.info/feed/A474136
Fetching AQI data from URL with ID: https://api.waqi.info/feed/A474136/?token=f8c9b0aad7e624da4286392f37a47c05f9322983
Fetching data for url: https://api.waqi.info/feed/A469291
Fetching AQI data from URL with ID: https://api.waqi.info/feed/A469291/?token=f8c9b0aad7e624da4286392f37a47c05f9322983
Fetching data for url: https://api.waqi.info/feed/A66394
Fetching AQI data from URL with ID: https://api.waqi.info/feed/A66394/?token=f8c9b0aad7e624da4286392f37a47c05f9322983


,pm25,country,city,street,date,url
0,10.0,Germany,Regensburg,Steyrerweg,2025-11-18,https://api.waqi.info/feed/A191545
0,8.0,Germany,Regensburg,Pfeilstraße,2025-11-18,https://api.waqi.info/feed/A474136
0,2.0,Germany,Regensburg,Hafnersteig,2025-11-18,https://api.waqi.info/feed/A469291
0,4.0,Germany,Regensburg,Fichtelgebirgstraße,2025-11-18,https://api.waqi.info/feed/A66394


In [19]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     4 non-null      float32       
 1   country  4 non-null      object        
 2   city     4 non-null      object        
 3   street   4 non-null      object        
 4   date     4 non-null      datetime64[ns]
 5   url      4 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 208.0+ bytes


In [20]:
# get existing data
df_aq = air_quality_fg.read()
# add lag_1, lag_2, lag_3 features
df_aq = df_aq.sort_values(by='date')


# calculate lag features for today's data
#aq_today_df["lag_1_pm25"] = df_aq["pm25"].iloc[-1]
#aq_today_df["lag_2_pm25"] = df_aq["pm25"].iloc[-2]
#aq_today_df["lag_3_pm25"] = df_aq["pm25"].iloc[-3]

for url in urls:
    temp_df = df_aq[df_aq['url'] == url].sort_values(by='date')
    aq_today_df.loc[aq_today_df['url'] == url, 'lag_1_pm25'] = temp_df["pm25"].iloc[-1]
    aq_today_df.loc[aq_today_df['url'] == url, 'lag_2_pm25'] = temp_df["pm25"].iloc[-2]
    aq_today_df.loc[aq_today_df['url'] == url, 'lag_3_pm25'] = temp_df["pm25"].iloc[-3]
aq_today_df




Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.09s) 


,pm25,country,city,street,date,url,lag_1_pm25,lag_2_pm25,lag_3_pm25
0,10.0,Germany,Regensburg,Steyrerweg,2025-11-18,https://api.waqi.info/feed/A191545,1.35,1.80,16.049999
0,8.0,Germany,Regensburg,Pfeilstraße,2025-11-18,https://api.waqi.info/feed/A474136,1.80,1.80,13.950000
0,2.0,Germany,Regensburg,Hafnersteig,2025-11-18,https://api.waqi.info/feed/A469291,0.25,1.83,2.900000
0,4.0,Germany,Regensburg,Fichtelgebirgstraße,2025-11-18,https://api.waqi.info/feed/A66394,0.47,0.42,5.600000


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [21]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 49.0°N 12.0°E
Elevation 353.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-18,2.65,0.0,4.829907,206.564987,Regensburg
1,2025-11-19,3.90,0.0,10.931203,162.758453,Regensburg
2,2025-11-20,3.35,0.0,12.096214,306.528961,Regensburg
3,2025-11-21,1.70,0.0,16.263872,27.699385,Regensburg
4,2025-11-22,1.10,0.0,8.699793,24.443953,Regensburg
5,2025-11-23,1.45,0.0,7.993298,97.765083,Regensburg
6,2025-11-24,-0.05,0.0,5.634891,116.564987,Regensburg


In [22]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         7 non-null      datetime64[ns]
 1   temperature_2m_mean          7 non-null      float32       
 2   precipitation_sum            7 non-null      float32       
 3   wind_speed_10m_max           7 non-null      float32       
 4   wind_direction_10m_dominant  7 non-null      float32       
 5   city                         7 non-null      object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 356.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [23]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2025-11-18 12:01:39,708 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279161/fs/1273929/fg/1737050


Uploading Dataframe: 100.00% |██████████| Rows 4/4 | Elapsed Time: 00:01 | Remaining Time: 00:00


(Job('air_quality_a_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 768010
         }
       },
       "result": {
         "observed_value": 2.0,
         "element_count": 4,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-18T11:01:39.000706Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expecta

In [24]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-18 12:01:48,706 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279161/fs/1273929/fg/1721945


Uploading Dataframe: 100.00% |██████████| Rows 7/7 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_a_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279161/jobs/named/weather_a_1_offline_fg_materialization/executions
2025-11-18 12:02:05,098 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-18 12:02:08,336 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-18 12:04:01,752 INFO: Waiting for execution to finish. Current state: FINISHED. Final status: SUCCEEDED
2025-11-18 12:04:02,214 INFO: Waiting for log aggregation to finish.
2025-11-18 12:04:02,216 INFO: Execution finished successfully.


(Job('weather_a_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 762131
         }
       },
       "result": {
         "observed_value": 4.829906940460205,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-18T11:01:48.000706Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type"

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
